In [1]:
import pandas as pd
import numpy as np

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow.keras import layers

ModuleNotFoundError: No module named 'pandas'

In [2]:
titanic = pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/train.csv")
titanic.head()

,survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,1,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,1,female,35.0,1,0,53.1000,First,C,Southampton,n
4,0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y


In [3]:
titanic_features = titanic.copy()
titanic_labels = titanic_features.pop('survived')

In [4]:
inputs = {}

for name, column in titanic_features.items():
  dtype = column.dtype
  if dtype == object:
    dtype = tf.string
  else:
    dtype = tf.float32

  inputs[name] = tf.keras.Input(shape=(1,), name=name, dtype=dtype)

inputs

{'sex': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'sex')>,
 'age': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'age')>,
 'n_siblings_spouses': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'n_siblings_spouses')>,
 'parch': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'parch')>,
 'fare': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'fare')>,
 'class': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'class')>,
 'deck': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'deck')>,
 'embark_town': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'embark_town')>,
 'alone': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'alone')>}

In [6]:
numeric_inputs.keys()

dict_keys(['age', 'n_siblings_spouses', 'parch', 'fare'])

In [5]:
numeric_inputs = {name:input for name,input in inputs.items()
                  if input.dtype==tf.float32}

x = layers.Concatenate()(list(numeric_inputs.values()))
norm = layers.Normalization()
norm.adapt(np.array(titanic[numeric_inputs.keys()]))
all_numeric_inputs = norm(x)

all_numeric_inputs

<KerasTensor: shape=(None, 4) dtype=float32 (created by layer 'normalization')>

In [6]:
preprocessed_inputs = [all_numeric_inputs]

In [7]:
for name, input in inputs.items():
  if input.dtype == tf.float32:
    continue

  lookup = layers.StringLookup(vocabulary=np.unique(titanic_features[name]))
  one_hot = layers.CategoryEncoding(max_tokens=lookup.vocab_size())

  x = lookup(input)
  x = one_hot(x)
  preprocessed_inputs.append(x)

In [9]:
preprocessed_inputs

[<KerasTensor: shape=(None, 4) dtype=float32 (created by layer 'normalization')>,
 <KerasTensor: shape=(None, 3) dtype=float32 (created by layer 'category_encoding')>,
 <KerasTensor: shape=(None, 4) dtype=float32 (created by layer 'category_encoding_1')>,
 <KerasTensor: shape=(None, 9) dtype=float32 (created by layer 'category_encoding_2')>,
 <KerasTensor: shape=(None, 5) dtype=float32 (created by layer 'category_encoding_3')>,
 <KerasTensor: shape=(None, 3) dtype=float32 (created by layer 'category_encoding_4')>]

In [ ]:
preprocessed_inputs_cat = layers.Concatenate()(preprocessed_inputs)

titanic_preprocessing = tf.keras.Model(inputs, preprocessed_inputs_cat)



In [14]:
help(layers.Add)

Help on class Add in module keras.layers.merging.add:

class Add(keras.layers.merging.base_merge._Merge)
 |  Add(*args, **kwargs)
 |  
 |  Layer that adds a list of inputs.
 |  
 |  It takes as input a list of tensors,
 |  all of the same shape, and returns
 |  a single tensor (also of the same shape).
 |  
 |  Examples:
 |  
 |  >>> input_shape = (2, 3, 4)
 |  >>> x1 = tf.random.normal(input_shape)
 |  >>> x2 = tf.random.normal(input_shape)
 |  >>> y = tf.keras.layers.Add()([x1, x2])
 |  >>> print(y.shape)
 |  (2, 3, 4)
 |  
 |  Used in a functional model:
 |  
 |  >>> input1 = tf.keras.layers.Input(shape=(16,))
 |  >>> x1 = tf.keras.layers.Dense(8, activation='relu')(input1)
 |  >>> input2 = tf.keras.layers.Input(shape=(32,))
 |  >>> x2 = tf.keras.layers.Dense(8, activation='relu')(input2)
 |  >>> # equivalent to `added = tf.keras.layers.add([x1, x2])`
 |  >>> added = tf.keras.layers.Add()([x1, x2])
 |  >>> out = tf.keras.layers.Dense(4)(added)
 |  >>> model = tf.keras.models.Model(i

In [7]:
titanic_features_dict = {name: np.array(value) 
                         for name, value in titanic_features.items()}

In [8]:
titanic_features

,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,female,35.0,1,0,53.1000,First,C,Southampton,n
4,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
...,...,...,...,...,...,...,...,...,...
622,male,28.0,0,0,10.5000,Second,unknown,Southampton,y
623,male,25.0,0,0,7.0500,Third,unknown,Southampton,y
624,female,19.0,0,0,30.0000,First,B,Southampton,y
625,female,28.0,1,2,23.4500,Third,unknown,Southampton,n


In [9]:
features_dict = {name:values[:1] for name, values in titanic_features_dict.items()}
titanic_preprocessing(features_dict)

NameError: name 'titanic_preprocessing' is not defined

In [17]:
features_dict

{'sex': array(['male'], dtype=object),
 'age': array([22.]),
 'n_siblings_spouses': array([1], dtype=int64),
 'parch': array([0], dtype=int64),
 'fare': array([7.25]),
 'class': array(['Third'], dtype=object),
 'deck': array(['unknown'], dtype=object),
 'embark_town': array(['Southampton'], dtype=object),
 'alone': array(['n'], dtype=object)}

In [18]:
def titanic_model(preprocessing_head, inputs):
  body = tf.keras.Sequential([
    layers.Dense(64),
    layers.Dense(1)
  ])

  preprocessed_inputs = preprocessing_head(inputs)
  result = body(preprocessed_inputs)
  model = tf.keras.Model(inputs, result)

  model.compile(loss=tf.losses.BinaryCrossentropy(from_logits=True),
                optimizer=tf.optimizers.Adam())
  return model

titanic_model = titanic_model(titanic_preprocessing, inputs)

In [20]:
titanic_model.fit(x=titanic_features_dict, y=titanic_labels, epochs=10)

Epoch 1/10
20/20 [==============================] - 1s 2ms/step - loss: 0.5967
Epoch 2/10
20/20 [==============================] - 0s 2ms/step - loss: 0.5177
Epoch 3/10
20/20 [==============================] - 0s 1ms/step - loss: 0.4771
Epoch 4/10
20/20 [==============================] - 0s 1ms/step - loss: 0.4550
Epoch 5/10
20/20 [==============================] - 0s 1ms/step - loss: 0.4414
Epoch 6/10
20/20 [==============================] - 0s 1ms/step - loss: 0.4318
Epoch 7/10
20/20 [==============================] - 0s 2ms/step - loss: 0.4276
Epoch 8/10
20/20 [==============================] - 0s 1ms/step - loss: 0.4243
Epoch 9/10
20/20 [==============================] - 0s 2ms/step - loss: 0.4225
Epoch 10/10
20/20 [==============================] - 0s 2ms/step - loss: 0.4221


In [21]:
titanic_model.save('titanic_model')


INFO:tensorflow:Assets written to: titanic_model\assets


INFO:tensorflow:Assets written to: titanic_model\assets


In [ ]:
import tensorflow as tf
import tf2onnx

model = tf.keras.models.load_model('titanic_model')

tf2onnx.convert.from_keras(model, output_path='titanic_model.onnx')

In [23]:
import onnxruntime as rt
import numpy as np

session = rt.InferenceSession('titanic_model.onnx')

In [30]:
inputDetails = session.get_inputs()

In [36]:
inputDetails[0].name

'age'

In [40]:
input_array = {'sex': np.array([['male']], dtype=str),
               'age': np.array([[22.]],dtype=np.float32),
               'n_siblings_spouses': np.array([[1]], dtype=np.float32),
                'parch': np.array([[0]], dtype=np.float32),
                'fare': np.array([[7.25]],dtype=np.float32),
                'class': np.array([['Third']], dtype=str),
                'deck': np.array([['unknown']], dtype=str),
                'embark_town': np.array([['Southampton']], dtype=str),
                'alone': np.array([['n']],dtype=str)}

In [37]:
np.array([['male']], dtype=str)

array([['male']], dtype='<U4')

In [24]:
features_dict

In [27]:
features_dict = {name:values[:1] for name, values in titanic_features_dict.items()}

In [43]:
before = titanic_model(features_dict)

In [44]:
before

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-1.912]], dtype=float32)>

In [41]:
y = session.run(None, input_array)

In [42]:
y

[array([[-1.912]], dtype=float32)]

In [28]:
input_array = {'sex': array(['male'], dtype=str),
 'age': array([22.]),
 'n_siblings_spouses': array([1], dtype=int64),
 'parch': array([0], dtype=int64),
 'fare': array([7.25]),
 'class': array(['Third'], dtype=object),
 'deck': array(['unknown'], dtype=object),
 'embark_town': array(['Southampton'], dtype=object),
 'alone': array(['n'], dtype=object)}

{'sex': array(['male'], dtype=object),
 'age': array([22.]),
 'n_siblings_spouses': array([1], dtype=int64),
 'parch': array([0], dtype=int64),
 'fare': array([7.25]),
 'class': array(['Third'], dtype=object),
 'deck': array(['unknown'], dtype=object),
 'embark_town': array(['Southampton'], dtype=object),
 'alone': array(['n'], dtype=object)}